In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

In [2]:
#Simulation
#Misura di tutti i qubit
n=3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i])
    


#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
print(circuits[-1].parameters)
circuits[-1].draw(output='mpl')

#circuits[1].draw()


job = execute(circ,
            backend=BasicAer.get_backend('qasm_simulator'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range])

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/1024*c.get('000', 0), counts2)))

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
set()


Counts
[{'000': 1024}, {'000': 160, '001': 864}, {'000': 525, '001': 499}, {'000': 864, '001': 160}, {'001': 1024}, {'000': 870, '001': 154}, {'000': 552, '001': 472}, {'000': 157, '001': 867}]


Counts of 000
[1.0, 0.15625, 0.5126953125, 0.84375, 0.0, 0.849609375, 0.5390625, 0.1533203125]


In [3]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmqx2'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts2)))
job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Job Status: job has successfully run


Counts
[{'110': 57, '011': 371, '111': 69, '000': 5682, '101': 72, '001': 1357, '010': 498, '100': 86}, {'110': 55, '011': 346, '111': 48, '000': 727, '101': 70, '001': 6365, '010': 497, '100': 84}, {'110': 55, '011': 340, '111': 48, '000': 5509, '101': 182, '001': 1532, '010': 423, '100': 103}, {'110': 56, '011': 399, '111': 68, '000': 3646, '101': 328, '001': 3020, '010': 424, '100': 251}, {'110': 89, '011': 405, '111': 65, '000': 1424, '101': 386, '001': 5026, '010': 403, '100': 394}, {'110': 88, '011': 413, '111': 75, '000': 5942, '101': 460, '001': 496, '010': 338, '100': 380}, {'110': 84, '011': 371, '111': 67, '000': 1526, '101': 267, '001': 5116, '010': 361, '100': 400}, {'110': 92, '011': 398, '111': 64, '000': 3716, '101': 145, '001': 3154, '010': 398, '100': 225}]


Counts of 000
[0.69360

In [4]:
#Run in real quantum computer - Transpile
#Misura di tutti i qubit
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

backend = IBMQ.get_backend('ibmqx2')
trans_ghz = transpile(circ, backend)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Job Status: job has successfully run


Counts
[{'110': 72, '011': 371, '111': 63, '000': 5758, '101': 73, '001': 1333, '010': 446, '100': 76}, {'110': 58, '011': 376, '111': 40, '000': 770, '101': 57, '001': 6361, '010': 444, '100': 86}, {'110': 59, '011': 351, '111': 35, '000': 5522, '101': 199, '001': 1495, '010': 406, '100': 125}, {'110': 78, '011': 400, '111': 77, '000': 3754, '101': 319, '001': 2941, '010': 361, '100': 262}, {'110': 59, '011': 449, '111': 65, '000': 1381, '101': 430, '001': 5098, '010': 315, '100': 395}, {'110': 62, '011': 400, '111': 62, '000': 5938, '101': 427, '001': 561, '010': 330, '100': 412}, {'110': 65, '011': 387, '111': 68, '000': 1466, '101': 266, '001': 5216, '010': 370, '100': 354}, {'110': 76, '011': 382, '111': 72, '000': 3665, '101': 141, '001': 3241, '010': 403, '100': 212}]


Counts of 000
[0.70288

In [5]:
#Run in real quantum computer - Transpile user
#Misura di tutti i qubit
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 2, q[1]: 1, q[2]: 0}

backend = IBMQ.get_backend('ibmqx2')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range], shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Job Status: job has successfully run


Counts
[{'110': 96, '011': 372, '111': 85, '000': 5637, '101': 82, '001': 1348, '010': 498, '100': 74}, {'110': 87, '011': 355, '111': 76, '000': 743, '101': 70, '001': 6254, '010': 524, '100': 83}, {'110': 90, '011': 386, '111': 57, '000': 5419, '101': 195, '001': 1454, '010': 459, '100': 132}, {'110': 82, '011': 443, '111': 85, '000': 3626, '101': 327, '001': 2893, '010': 433, '100': 303}, {'110': 108, '011': 462, '111': 84, '000': 1400, '101': 413, '001': 4929, '010': 380, '100': 416}, {'110': 116, '011': 450, '111': 116, '000': 5798, '101': 404, '001': 493, '010': 409, '100': 406}, {'110': 108, '011': 424, '111': 104, '000': 1457, '101': 261, '001': 5114, '010': 362, '100': 362}, {'110': 113, '011': 413, '111': 89, '000': 3567, '101': 138, '001': 3165, '010': 461, '100': 246}]


Counts of 000
[0